In [14]:
# %pip install backtesting pandas numpy

## Data

In [15]:
import pandas as pd
from datetime import datetime, timedelta

AAPL = pd.read_csv('price/raw/AAPL.csv')
start_date = datetime(2014, 1, 1)
end_date = datetime(2015, 12, 31)
AAPL['Date'] = pd.to_datetime(AAPL['Date'])  # Convert 'Date' column to datetime format
AAPL = AAPL[(AAPL['Date'] >= start_date) & (AAPL['Date'] <= end_date)]  # Filter AAPL data based on start and end dates
AAPL

,Date,Open,High,Low,Close,Adj Close,Volume
333,2014-01-02,79.382858,79.575714,78.860001,79.018570,73.522530,58671200
334,2014-01-03,78.980003,79.099998,77.204285,77.282860,71.907555,98116900
335,2014-01-06,76.778572,78.114288,76.228569,77.704285,72.299644,103152700
336,2014-01-07,77.760002,77.994286,76.845711,77.148575,71.782608,79302300
337,2014-01-08,76.972855,77.937141,76.955711,77.637146,72.237190,64632400
...,...,...,...,...,...,...,...
832,2015-12-24,109.000000,109.000000,107.949997,108.029999,104.380112,13570400
833,2015-12-28,107.589996,107.690002,106.180000,106.820000,103.210999,26704200
834,2015-12-29,106.959999,109.430000,106.860001,108.739998,105.066116,30931200
835,2015-12-30,108.580002,108.699997,107.180000,107.320000,103.694107,25213800


## Strategies

1. SMA

In [16]:
import pandas as pd


def SMA(values, n):
    """
    Return simple moving average of `values`, at
    each step taking into account `n` previous values.
    """
    return pd.Series(values).rolling(n).mean()


from backtesting import Strategy
from backtesting.lib import crossover


class SmaCross(Strategy):
    # Define the two MA lags as *class variables*
    # for later optimization
    n1 = 10
    n2 = 20

    def init(self):
        # Precompute the two moving averages
        self.sma1 = self.I(SMA, self.data.Close, self.n1)
        self.sma2 = self.I(SMA, self.data.Close, self.n2)

    def next(self):
        # If sma1 crosses above sma2, close any existing
        # short trades, and buy the asset
        if crossover(self.sma1, self.sma2):
            self.position.close()
            self.buy()

        # Else, if sma1 crosses below sma2, close any existing
        # long trades, and sell the asset
        elif crossover(self.sma2, self.sma1):
            self.position.close()
            self.sell()

In [20]:
from backtesting import Backtest

bt = Backtest(AAPL, SmaCross, cash=10_000, commission=.002)
# stats = bt.run()
# stats

/opt/homebrew/Caskroom/miniforge/base/envs/dev-env/lib/python3.13/site-packages/backtesting/backtesting.py:1192: FutureWarning: Index.is_numeric is deprecated. Use pandas.api.types.is_any_real_numeric_dtype instead
  (data.index.is_numeric() and
/var/folders/yn/y7_5psd14q91td0_0lk976180000gn/T/ipykernel_20989/2482915356.py:3: UserWarning: Data index is not datetime. Assuming simple periods, but `pd.DateTimeIndex` is advised.
  bt = Backtest(AAPL, SmaCross, cash=10_000, commission=.002)


In [18]:
stats = bt.optimize(n1=range(5, 30, 5),
                    n2=range(10, 70, 5),
                    maximize='Equity Final [$]',
                    constraint=lambda param: param.n1 < param.n2)
stats

Start                                   333.0
End                                     836.0
Duration                                503.0
Exposure Time [%]                    86.70635
Equity Final [$]                  13493.98161
Equity Peak [$]                   16426.79794
Commissions [$]                     383.69241
Return [%]                           34.93982
Buy & Hold Return [%]                38.27388
Return (Ann.) [%]                         0.0
Volatility (Ann.) [%]                     NaN
Sharpe Ratio                              NaN
Sortino Ratio                             NaN
Calmar Ratio                              0.0
Max. Drawdown [%]                   -20.45959
Avg. Drawdown [%]                    -3.25642
Max. Drawdown Duration                  184.0
Avg. Drawdown Duration               19.66667
# Trades                                  7.0
Win Rate [%]                         57.14286
Best Trade [%]                       46.57095
Worst Trade [%]                   

In [19]:
bt.plot(plot_volume=False, plot_pl=False)

GridPlot(id='p1437', ...)